In [222]:
import pandas as pd
import re
from pandas import ExcelWriter
df = pd.read_excel('natasha.xls')
df = df[8:-1]
df.columns = ['date', 'text', 'del_1', 'bank', 'analitic',
              'del_2', 'sum', 'del_3', 'del_4', 'sum2',
              'del_5', 'del_6']

for i in range(1,7):
    df.drop(columns=[f'del_{i}'], inplace=True)

df['text'] = df['text'].str.lower()
def check(row):
    if 'перевод денежных средств по поручению физических лиц согласно реестру' in row:
        return False 
    else:
        return True

df['checker'] = df['text'].apply(check)
df = df[df['checker']]
df.drop(columns=['sum2', 'checker'], inplace=True)

In [223]:
from yargy import rule, or_, Parser
from yargy.predicates import gram

In [224]:
# RULEs FOR yargy-parser
FIRST = gram('Name')
LAST = gram('Surn')
MIDDLE = gram('Patr')
ABBR = gram('Abbr')
NAME = or_(
    rule(FIRST),
    rule(LAST),
    rule(FIRST, LAST),
    rule(LAST, FIRST),
    rule(FIRST, MIDDLE, LAST),
    rule(ABBR, '.', ABBR, LAST),
    rule(LAST, ABBR, '.', ABBR, '.')
)


In [225]:
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    PER,
    NamesExtractor,
    DatesExtractor,
    MoneyExtractor,
    AddrExtractor,
    Doc)
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)
names_extractor = NamesExtractor(morph_vocab)
dates_extractor = DatesExtractor(morph_vocab)
money_extractor = MoneyExtractor(morph_vocab)
addr_extractor = AddrExtractor(morph_vocab)


In [226]:
text = '''
Поступление на расчетный счет 0000-000083 от 21.02.2022 12:00:01
ЗА 18/02/2022;ПЛАТОНОВ ВЛАДИМИР СЕРГЕЕВИЧ;ГОДОВОЙ ЧЛЕНСКИЙ ВЗНОС по вх.д. 720604 от 18.02.2022'''


In [235]:

def cleaner(text):
    try:
        text = re.sub(r"\d+", "", text)
        symbols_to_change = ['\n','/',';','.',':',' по ']
        for symbol in symbols_to_change:
            text = text.replace(symbol, ' ')
        text = text.capitalize().strip()
    except:
        pass
    return text

In [233]:

def pars_name(text):
    try:
        result = ''
        parser = Parser(NAME)
        for match in parser.findall(text):
            start, stop = match.span
            result += text[start:stop] + ' '
        return result.strip()
    except:
        return text

In [234]:
def lemattize_name(row):
    doc = Doc(row)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    doc.parse_syntax(syntax_parser)
    doc.tag_ner(ner_tagger)

    for token in doc.tokens:
        token.lemmatize(morph_vocab)

    my_dict = {_.text: _.lemma for _ in doc.tokens[:]}
    res = ''
    for i in my_dict:
        res += my_dict.get(i).capitalize() + ' '
    return res.strip()

In [236]:
def lemmatize(row):
    if row['name']:
        return row['name']
    text = row['text']
    text = cleaner(text)
    text = pars_name(text)
    text = lemattize_name(text)
    return text


In [228]:
def ticket_number(row):
    pattern = r'\d+-77'
    try:
        return re.findall(pattern, row)[0]
    except IndexError:
        return ''

df['ticket'] = df['text'].apply(ticket_number)


In [229]:
def check_name(row: str):
    row = row.lower()
    pattern = r';\w+\s\w+\s\w+;'
    try:
        text:str = re.findall(pattern, row)[0]
    except IndexError:
        return ''
    text = text.replace(';', '')
    res = ''
    for i in text.split():
        res += i.capitalize() + ' '
    res = res.strip()
    return res


In [237]:
df['ticket'] = df['text'].apply(ticket_number)
df['name'] = df['text'].apply(check_name)
df['name'] = df.apply(lemmatize, axis=1)

C:\ProgramData\Anaconda3\envs\ds_3_7\lib\site-packages\pandas\core\series.py:4135: FutureWarning: 'is_extension_type' is deprecated and will be removed in a future version.  Use 'is_extension_array_dtype' instead.
  broadcast_axis=None,


In [238]:
with ExcelWriter('new_v4.xlsx') as writer:
    df.to_excel(writer)